In [ ]:
# 1) Imports (runs in your repo root)
from train_forecasting_model import (
    load_hourly_csv,
    build_supervised_dataset,
    build_pipeline,
    split_by_date_ranges,
)
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import pandas as pd

# 2) Load data
csv_path = "Solicitud_Historica/Historico_pm25_filtered.csv"
df_raw = load_hourly_csv(csv_path)

# 3) Build supervised matrix (choose horizon/max_lag)
target = "BAR-TORR"
horizon = 6
max_lag = 48
sup = build_supervised_dataset(
    df_raw=df_raw,
    target_col=target,
    horizon=horizon,
    max_lag=max_lag,
    missing_col_threshold=0.5,
)

# 4) Date-based splits (your requested windows)
splits = split_by_date_ranges(
    sup,
    train_start="2019-01-01",
    train_end="2021-12-31",
    test_start="2022-01-01",
    test_end="2023-12-31",
    val_start="2024-01-01",
    val_end="2024-12-31",
)
X_train, y_train = splits["train"]["X"], splits["train"]["y"]
X_test, y_test = splits["test"]["X"], splits["test"]["y"]
X_val, y_val = splits["val"]["X"], splits["val"]["y"]

# 5) Build model/pipeline and fit
pipe = build_pipeline(
    sup.numeric_cols,
    sup.categorical_cols,
    n_estimators=300,
    max_depth=10,
)
pipe.fit(X_train, y_train)

# 6) Evaluate
def eval_block(X, y, name):
    if len(X) == 0:
        print(f"{name}: empty split")
        return
    preds = pipe.predict(X)
    mae = mean_absolute_error(y, preds)
    rmse = root_mean_squared_error(y, preds)
    r2 = r2_score(y, preds)
    print(f"{name} MAE: {mae:.2f} | RMSE: {rmse:.2f}, R2: {r2:.2f}")
    display(pd.DataFrame({"y_true": y[:5], "y_pred": preds[:5]}))

eval_block(X_train, y_train, "Train")
eval_block(X_test, y_test, "Test")
eval_block(X_val, y_val, "Val")


/home/manuel/Documents/Maestria_Analitica/AQI_Predict/air_quality_preprocessor_forecasting.py:74: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df = df.asfreq(self.freq)
/home/manuel/Documents/Maestria_Analitica/AQI_Predict/air_quality_preprocessor_forecasting.py:175: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{base_col}_lag{lag}"] = df[base_col].shift(lag)
/home/manuel/Documents/Maestria_Analitica/AQI_Predict/air_quality_preprocessor_forecasting.py:175: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame,

Train MAE: 3.52 | RMSE: 4.52, R2: 0.82


,y_true,y_pred
0,20.8951,17.533130
1,5.8579,15.161478
2,16.5909,16.992800
3,9.6026,13.913628
4,10.9169,14.738375


Test MAE: 4.25 | RMSE: 5.57, R2: 0.43


,y_true,y_pred
0,9.18079,27.553328
1,13.78030,28.432632
2,13.78640,23.652337
3,13.87040,22.711674
4,16.23860,16.423173


Val MAE: 4.88 | RMSE: 6.44, R2: 0.55


,y_true,y_pred
0,16.9649,27.663711
1,10.2521,25.224256
2,16.1059,25.448162
3,15.1242,22.149877
4,17.5271,18.510719
